# Astrometry

In [3]:
# make a table in LaTeX format for easy pasting

import h5py
import matplotlib.pyplot as plt
import numpy as np
from lenstronomy.Plots.model_plot import ModelPlot

# define filters and system names
names = ['J0407-5006', 'J0602-4335', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
filter = 'F814W'

# table format:
# Lens System Name, RA_lens, Dec_lens, RA_im1, Dec_im1, Ra_im2, Dec_im2, Image Separation [arcsec]
for i, name in enumerate(names):
    filename = f"../../cutout_data/{name}/{filter}/{name}_{filter}_final.hdf5"

    with h5py.File(filename, "r") as f:
        kwargs = {}
        for key in f:
            kwargs[key] = f[key][()]

    if name=='J0602-4335':
        ra_lens = kwargs['center_x_0']
        dec_lens = kwargs['center_y_0']        
    else:
        ra_lens = kwargs['center_x']
        dec_lens = kwargs['center_y']

    ra_im1 = kwargs['image_position'][0][0] - ra_lens
    dec_im1 = kwargs['image_position'][1][0] - dec_lens

    ra_im2 = kwargs['image_position'][0][1] - ra_lens
    dec_im2 = kwargs['image_position'][1][1] - dec_lens

    image_sep = np.sqrt((kwargs['image_position'][0][0] - kwargs['image_position'][0][1])**2 + (kwargs['image_position'][1][0] - kwargs['image_position'][1][1])**2)

    # print the data in LaTeX format
    print(f"{name} & {ra_lens:.3f} & {dec_lens:.3f} & {ra_im1:.3f} & {dec_im1:.3f} & {ra_im2:.3f} & {dec_im2:.3f} & {image_sep:.3f} \\\\")

J0407-5006 & 3.455 & -0.678 & -0.157 & -0.257 & 0.734 & 1.214 & 1.721 \\
J0602-4335 & 3.946 & 0.013 & -0.764 & -0.926 & 0.416 & 0.470 & 1.828 \\
J0806+2006 & 4.191 & -0.157 & -0.929 & 0.501 & 0.390 & -0.185 & 1.486 \\
J1442+4055 & 4.234 & -0.265 & -1.300 & 0.394 & 0.758 & -0.162 & 2.132 \\
J1515+1511 & 4.556 & -0.214 & -1.403 & 0.851 & 0.265 & -0.266 & 2.008 \\
J1620+1203 & 3.274 & -0.254 & -0.339 & -0.398 & 1.768 & 1.462 & 2.810 \\
J2325-5229 & 3.795 & -0.364 & -0.488 & 1.657 & 0.911 & -0.792 & 2.821 \\


# Magnitude Table

In [4]:
import h5py
import numpy as np
import os

# Define filters and system names
names_f814w = ['J0407-5006', 'J0602-4335', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
names_f475x = ['J0407-5006', 'J0602-4335', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']

def mag_sigma(flux, sigma_flux, zp_err_mag=0.018):
    sigma_flux = np.asarray(sigma_flux)
    sigma_upper = sigma_flux[0]
    sigma_lower = sigma_flux[1]

    # Convert zeropoint mag uncertainty to flux error
    zp_flux_err = 0.921 * flux * zp_err_mag  # ≈ ln(10) * 0.4

    # Total flux uncertainty (add in quadrature)
    total_upper = np.sqrt(sigma_upper**2 + zp_flux_err**2)
    total_lower = np.sqrt(sigma_lower**2 + zp_flux_err**2)

    # Convert flux error to magnitude error
    mag_err_upper = (2.5 / np.log(10)) * (total_upper / flux)
    mag_err_lower = (2.5 / np.log(10)) * (total_lower / flux)

    return mag_err_upper, mag_err_lower

# LaTeX table header
print("\\begin{tabular*}{\\textwidth}{@{\\extracolsep{\\fill}}lccc|ccc}")
print("\\hline")
print("\\multicolumn{4}{c|}{F814W Magnitude} & \\multicolumn{3}{c}{F475X Magnitude} \\\\")
print("\\hline")
print("Lens System & Lens & Image A & Image B & Lens & Image A & Image B \\\\")
print("\\hline")

# Table content
for name in names_f814w:
    # F814W file
    filename_f814w = f"../../cutout_data/{name}/F814W/{name}_F814W_final.hdf5"
    with h5py.File(filename_f814w, "r") as f:
        kwargs_f814w = {key: f[key][()] for key in f}

    # Try to open F475X file if it exists
    filename_f475x = f"../../cutout_data/{name}/F475X/{name}_F475X_final.hdf5"
    has_f475x = os.path.exists(filename_f475x)
    if has_f475x:
        with h5py.File(filename_f475x, "r") as f:
            kwargs_f475x = {key: f[key][()] for key in f}

    # --- F814W ---
    lens_flux = kwargs_f814w['lens_flux']
    lens_flux_sigma = kwargs_f814w['lens_flux_sigma']
    lens_mag = kwargs_f814w['lens_mag']
    lens_mag_sigmas = mag_sigma(lens_flux, lens_flux_sigma)

    im1_flux = kwargs_f814w['image_fluxes'][0]
    im1_flux_sigma = kwargs_f814w['image_fluxes_sigma'][0]
    im1_mag = kwargs_f814w['image_mags'][0]
    im1_mag_sigmas = mag_sigma(im1_flux, im1_flux_sigma)

    im2_flux = kwargs_f814w['image_fluxes'][1]
    im2_flux_sigma = kwargs_f814w['image_fluxes_sigma'][1]
    im2_mag = kwargs_f814w['image_mags'][1]
    im2_mag_sigmas = mag_sigma(im2_flux, im2_flux_sigma)

    # --- F475X (if available) ---
    if has_f475x:
        lens_flux_f475x = kwargs_f475x['lens_flux']
        lens_flux_sigma_f475x = kwargs_f475x['lens_flux_sigma']
        lens_mag_f475x = kwargs_f475x['lens_mag']
        lens_mag_sigmas_f475x = mag_sigma(lens_flux_f475x, lens_flux_sigma_f475x)

        im1_flux_f475x = kwargs_f475x['image_fluxes'][0]
        im1_flux_sigma_f475x = kwargs_f475x['image_fluxes_sigma'][0]
        im1_mag_f475x = kwargs_f475x['image_mags'][0]
        im1_mag_sigmas_f475x = mag_sigma(im1_flux_f475x, im1_flux_sigma_f475x)

        im2_flux_f475x = kwargs_f475x['image_fluxes'][1]
        im2_flux_sigma_f475x = kwargs_f475x['image_fluxes_sigma'][1]
        im2_mag_f475x = kwargs_f475x['image_mags'][1]
        im2_mag_sigmas_f475x = mag_sigma(im2_flux_f475x, im2_flux_sigma_f475x)

        f475x_lens = f"${lens_mag_f475x:.2f}^{{+{lens_mag_sigmas_f475x[0]:.2f}}}_{{-{lens_mag_sigmas_f475x[1]:.2f}}}$"
        f475x_im1 = f"${im1_mag_f475x:.2f}^{{+{im1_mag_sigmas_f475x[0]:.2f}}}_{{-{im1_mag_sigmas_f475x[1]:.2f}}}$"
        f475x_im2 = f"${im2_mag_f475x:.2f}^{{+{im2_mag_sigmas_f475x[0]:.2f}}}_{{-{im2_mag_sigmas_f475x[1]:.2f}}}$"
    else:
        f475x_lens = f475x_im1 = f475x_im2 = "--"

    # --- Print table row ---
    print(f"{name} & "
          f"${lens_mag:.2f}^{{+{lens_mag_sigmas[0]:.2f}}}_{{-{lens_mag_sigmas[1]:.2f}}}$ & "
          f"${im1_mag:.2f}^{{+{im1_mag_sigmas[0]:.2f}}}_{{-{im1_mag_sigmas[1]:.2f}}}$ & "
          f"${im2_mag:.2f}^{{+{im2_mag_sigmas[0]:.2f}}}_{{-{im2_mag_sigmas[1]:.2f}}}$ & "
          f"{f475x_lens} & {f475x_im1} & {f475x_im2} \\\\")

print("\\hline")
print("\\end{tabular*}")


\begin{tabular*}{\textwidth}{@{\extracolsep{\fill}}lccc|ccc}
\hline
\multicolumn{4}{c|}{F814W Magnitude} & \multicolumn{3}{c}{F475X Magnitude} \\
\hline
Lens System & Lens & Image A & Image B & Lens & Image A & Image B \\
\hline
J0407-5006 & $20.35^{+0.02}_{-0.02}$ & $19.39^{+0.02}_{-0.02}$ & $18.01^{+0.02}_{-0.02}$ & $22.39^{+0.02}_{-0.02}$ & $19.81^{+0.02}_{-0.02}$ & $18.66^{+0.02}_{-0.02}$ \\
J0602-4335 & $19.01^{+0.02}_{-0.02}$ & $19.66^{+0.02}_{-0.02}$ & $19.46^{+0.02}_{-0.02}$ & $20.80^{+0.02}_{-0.02}$ & $19.87^{+0.02}_{-0.02}$ & $19.69^{+0.02}_{-0.02}$ \\
J0806+2006 & $19.88^{+0.02}_{-0.02}$ & $19.25^{+0.02}_{-0.02}$ & $20.07^{+0.02}_{-0.02}$ & $23.02^{+0.13}_{-0.25}$ & $20.02^{+0.02}_{-0.02}$ & $20.73^{+0.02}_{-0.02}$ \\
J1442+4055 & $19.02^{+0.02}_{-0.02}$ & $18.16^{+0.02}_{-0.02}$ & $18.99^{+0.02}_{-0.02}$ & $20.80^{+0.02}_{-0.02}$ & $18.88^{+0.02}_{-0.02}$ & $19.82^{+0.02}_{-0.02}$ \\
J1515+1511 & $21.36^{+0.02}_{-0.02}$ & $18.17^{+0.02}_{-0.02}$ & $18.58^{+0.02}_{-0.02}$ & 

# Lens Light Parameter Table

In [17]:
import pickle
import numpy as np
import h5py

names_f814w = ['J0407-5006', 'J0602-4335', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
names_f475x = ['J0407-5006', 'J0602-4335', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']

# LaTeX header
print("\\begin{table*}[htb]")
print("\\centering")
print("\\caption{Median lens light parameters computed from the best-fit model. The associated uncertainties are statistical and were calculated using the 84th and 16th percentiles.}")
print("\\label{tab:lens_light_params}")
print("\\begin{tabular*}{\\textwidth}{@{\\extracolsep{\\fill}}lcccc|cccc}")
print("\\hline")
print("\\multicolumn{5}{c|}{F814W} & \\multicolumn{4}{c}{F475X} \\\\")
print("\\hline")
print("Lens System & $R_{\\text{S\\'ersic}}$ [arcsec] & $n_{\\text{S\\'ersic}}$ & $q$ & $\\phi_{\\text{ext}}$ [deg] & "
      "$R_{\\text{S\\'ersic}}$ [arcsec] & $n_{\\text{S\\'ersic}}$ & $q$ & $\\phi_{\\text{ext}}$ [deg] \\\\")
print("\\hline")

for i, name in enumerate(names_f814w):
    filename_f814w = f"../../cutout_data/{name}/F814W/{name}_F814W.pkl"
    with open(filename_f814w, "rb") as f:
        loaded_data_f814w = pickle.load(f)

    chain_list_f814w = loaded_data_f814w.get('chain_list')
    sampler_type, samples_mcmc_f814w, param_mcmc_f814w, dist_mcmc_f814w = chain_list_f814w[1]

    # ------ Special Handling for J0602 ------
    n_profiles = 2 if name == "J0602-4335" else 1
    for prof_idx in range(n_profiles):
        param_names = [
            f'R_sersic_lens_light{prof_idx}', 
            f'n_sersic_lens_light{prof_idx}', 
            f'e1_lens_light{prof_idx}', 
            f'e2_lens_light{prof_idx}'
        ]
        indices_f814w = [param_mcmc_f814w.index(p) for p in param_names]

        r_sersic_chain_f814w = samples_mcmc_f814w[:, indices_f814w[0]]
        n_sersic_chain_f814w = samples_mcmc_f814w[:, indices_f814w[1]]
        e1_chain_f814w = samples_mcmc_f814w[:, indices_f814w[2]]
        e2_chain_f814w = samples_mcmc_f814w[:, indices_f814w[3]]

        e_chain_f814w = np.sqrt(e1_chain_f814w**2 + e2_chain_f814w**2)
        q_chain_f814w = (1 - e_chain_f814w) / (1 + e_chain_f814w)
        phi_chain = 0.5 * np.arctan2(e2_chain_f814w, e1_chain_f814w) * 180 / np.pi

        def get_median_and_uncertainties(samples):
            median = np.percentile(samples, 50)
            lower = median - np.percentile(samples, 16)
            upper = np.percentile(samples, 84) - median
            return median, lower, upper

        r_median_f814w, r_lower_f814w, r_upper_f814w = get_median_and_uncertainties(r_sersic_chain_f814w)
        n_median_f814w, n_lower_f814w, n_upper_f814w = get_median_and_uncertainties(n_sersic_chain_f814w)
        q_median_f814w, q_lower_f814w, q_upper_f814w = get_median_and_uncertainties(q_chain_f814w)
        phi_median_f814w, phi_lower_f814w, phi_upper_f814w = get_median_and_uncertainties(phi_chain)

        # --- F475X for each light profile ---
        if name in names_f475x:
            filename_f475x = f"../../cutout_data/{name}/F475X/{name}_F475X.pkl"
            with open(filename_f475x, "rb") as f:
                loaded_data_f475x = pickle.load(f)

            chain_list_f475x = loaded_data_f475x.get('chain_list')
            sampler_type, samples_mcmc_f475x, param_mcmc_f475x, dist_mcmc_f475x = chain_list_f475x[1]
            indices_f475x = [param_mcmc_f475x.index(p) for p in param_names]

            r_sersic_chain_f475x = samples_mcmc_f475x[:, indices_f475x[0]]
            n_sersic_chain_f475x = samples_mcmc_f475x[:, indices_f475x[1]]
            e1_chain_f475x = samples_mcmc_f475x[:, indices_f475x[2]]
            e2_chain_f475x = samples_mcmc_f475x[:, indices_f475x[3]]

            e_chain_f475x = np.sqrt(e1_chain_f475x**2 + e2_chain_f475x**2)
            q_chain_f475x = (1 - e_chain_f475x) / (1 + e_chain_f475x)
            phi_chain_f475x = 0.5 * np.arctan2(e2_chain_f475x, e1_chain_f475x) * 180 / np.pi

            r_median_f475x, r_lower_f475x, r_upper_f475x = get_median_and_uncertainties(r_sersic_chain_f475x)
            n_median_f475x, n_lower_f475x, n_upper_f475x = get_median_and_uncertainties(n_sersic_chain_f475x)
            q_median_f475x, q_lower_f475x, q_upper_f475x = get_median_and_uncertainties(q_chain_f475x)
            phi_median_f475x, phi_lower_f475x, phi_upper_f475x = get_median_and_uncertainties(phi_chain_f475x)

            r_str_f475x = f"${r_median_f475x:.2f}^{{+{r_upper_f475x:.2f}}}_{{-{r_lower_f475x:.2f}}}$"
            n_str_f475x = f"${n_median_f475x:.2f}^{{+{n_upper_f475x:.2f}}}_{{-{n_lower_f475x:.2f}}}$"
            q_str_f475x = f"${q_median_f475x:.3f}^{{+{q_upper_f475x:.3f}}}_{{-{q_lower_f475x:.3f}}}$"
            phi_str_f475x = f"${phi_median_f475x:.1f}^{{+{phi_upper_f475x:.1f}}}_{{-{phi_lower_f475x:.1f}}}$"
        else:
            r_str_f475x = "$--$"
            n_str_f475x = "$--$"
            q_str_f475x = "$--$"
            phi_str_f475x = "$--$"

        # Modify name for second profile to distinguish it
        name_display = name if prof_idx == 0 else f"--"

        # Print row
        print(f"{name_display} & "
              f"${r_median_f814w:.2f}^{{+{r_upper_f814w:.2f}}}_{{-{r_lower_f814w:.2f}}}$ & "
              f"${n_median_f814w:.2f}^{{+{n_upper_f814w:.2f}}}_{{-{n_lower_f814w:.2f}}}$ & "
              f"${q_median_f814w:.3f}^{{+{q_upper_f814w:.3f}}}_{{-{q_lower_f814w:.3f}}}$ & "
              f"${phi_median_f814w:.1f}^{{+{phi_upper_f814w:.1f}}}_{{-{phi_lower_f814w:.1f}}}$ & "
              f"{r_str_f475x} & {n_str_f475x} & {q_str_f475x} & {phi_str_f475x} \\\\")

print("\\hline")
print("\\end{tabular*}")
print("\\end{table*}")

\begin{table*}[htb]
\centering
\caption{Median lens light parameters computed from the best-fit model. The associated uncertainties are statistical and were calculated using the 84th and 16th percentiles.}
\label{tab:lens_light_params}
\begin{tabular*}{\textwidth}{@{\extracolsep{\fill}}lcccc|cccc}
\hline
\multicolumn{5}{c|}{F814W} & \multicolumn{4}{c}{F475X} \\
\hline
Lens System & $R_{\text{S\'ersic}}$ [arcsec] & $n_{\text{S\'ersic}}$ & $q$ & $\phi_{\text{ext}}$ [deg] & $R_{\text{S\'ersic}}$ [arcsec] & $n_{\text{S\'ersic}}$ & $q$ & $\phi_{\text{ext}}$ [deg] \\
\hline
J0407-5006 & $0.40^{+0.01}_{-0.01}$ & $4.85^{+0.15}_{-0.13}$ & $0.664^{+0.009}_{-0.009}$ & $-8.3^{+0.9}_{-0.9}$ & $9.92^{+0.06}_{-0.14}$ & $5.96^{+0.03}_{-0.05}$ & $0.795^{+0.044}_{-0.042}$ & $-2.8^{+8.4}_{-6.8}$ \\
J0602-4335 & $0.62^{+0.00}_{-0.00}$ & $0.81^{+0.02}_{-0.02}$ & $0.110^{+0.001}_{-0.001}$ & $-46.0^{+0.1}_{-0.1}$ & $0.65^{+0.02}_{-0.01}$ & $0.73^{+0.04}_{-0.03}$ & $0.089^{+0.003}_{-0.003}$ & $-45.9^{+0.1}_{-

# Lens MASS

In [15]:
import pickle
import numpy as np
import os

names_f814w = ['J0407-5006', 'J0602-4335', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
names_f475x = ['J0407-5006', 'J0602-4335', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']

# LaTeX table header
print("\\begin{table*}[htb]")
print("\\centering")
print("\\caption{Median lens mass parameters computed from the best-fit model. The associated uncertainties are statistical and were calculated using the 84th and 16th percentiles.}")
print("\\label{tab:lens_mass_params}")
print("\\begin{tabular*}{\\textwidth}{@{\\extracolsep{\\fill}}lcccc|cccc}")
print("\\hline")
print("\\multicolumn{5}{c|}{{F814W}} & \\multicolumn{4}{c}{{F475X}} \\\\")
print("\\hline")
print("Lens System & $\\theta_E$ [arcsec] & $q_{\\text{mass}}$ & $\\phi_{\\text{mass}}$ [deg] & $\\gamma$ & "
      "$\\theta_E$ & $q_{\\text{mass}}$ & $\\phi_{\\text{mass}}$ [deg] & $\\gamma$ \\\\")
print("\\hline")

def get_median_and_uncertainties(samples):
    median = np.percentile(samples, 50)
    lower = median - np.percentile(samples, 16)
    upper = np.percentile(samples, 84) - median
    return median, lower, upper

for name in names_f814w:
    filename_f814w = f"../../cutout_data/{name}/F814W/{name}_F814W.pkl"
    filename_f475x = f"../../cutout_data/{name}/F475X/{name}_F475X.pkl"

    # F814W
    with open(filename_f814w, "rb") as f:
        loaded_data_f814w = pickle.load(f)
    chain_list_f814w = loaded_data_f814w.get('chain_list')
    sampler_type, samples_mcmc, param_mcmc, dist_mcmc = chain_list_f814w[1]

    param_names_mass = ['theta_E_lens0', 'e1_lens0', 'e2_lens0', 'gamma1_lens1', 'gamma2_lens1']
    indices_f814w = [param_mcmc.index(p) for p in param_names_mass]

    theta_E_chain = samples_mcmc[:, indices_f814w[0]]
    e1_chain = samples_mcmc[:, indices_f814w[1]]
    e2_chain = samples_mcmc[:, indices_f814w[2]]
    gamma1_chain = samples_mcmc[:, indices_f814w[3]]
    gamma2_chain = samples_mcmc[:, indices_f814w[4]]

    e_chain = np.sqrt(e1_chain**2 + e2_chain**2)
    q_mass_chain = (1 - e_chain) / (1 + e_chain)

    gamma_chain = np.sqrt(gamma1_chain**2 + gamma2_chain**2)
    phi_ext_chain = 0.5 * np.arctan2(e2_chain, e1_chain) * 180 / np.pi

    theta_E_median, theta_E_lower, theta_E_upper = get_median_and_uncertainties(theta_E_chain)
    q_mass_median, q_mass_lower, q_mass_upper = get_median_and_uncertainties(q_mass_chain)
    gamma_median, gamma_lower, gamma_upper = get_median_and_uncertainties(gamma_chain)
    phi_ext_median, phi_ext_lower, phi_ext_upper = get_median_and_uncertainties(phi_ext_chain)

    # Format values
    theta_E_f814w = f"${theta_E_median:.2f}^{{+{theta_E_upper:.2f}}}_{{-{theta_E_lower:.2f}}}$"
    q_mass_f814w = f"${q_mass_median:.2f}^{{+{q_mass_upper:.2f}}}_{{-{q_mass_lower:.2f}}}$"
    gamma_f814w = f"${gamma_median:.2f}^{{+{gamma_upper:.2f}}}_{{-{gamma_lower:.2f}}}$"
    phi_ext_f814w = f"${phi_ext_median:.1f}^{{+{phi_ext_upper:.1f}}}_{{-{phi_ext_lower:.1f}}}$"

    # Default values for F475X
    theta_E_f475x = q_mass_f475x = gamma_f475x = phi_ext_f475x = '...'

    if name in names_f475x and os.path.exists(filename_f475x):
        with open(filename_f475x, "rb") as f:
            loaded_data_f475x = pickle.load(f)
        chain_list_f475x = loaded_data_f475x.get('chain_list')
        sampler_type, samples_mcmc, param_mcmc, dist_mcmc = chain_list_f475x[1]

        indices_f475x = [param_mcmc.index(p) for p in param_names_mass]

        theta_E_chain_f475x = samples_mcmc[:, indices_f475x[0]]
        e1_chain_f475x = samples_mcmc[:, indices_f475x[1]]
        e2_chain_f475x = samples_mcmc[:, indices_f475x[2]]
        gamma1_chain_f475x = samples_mcmc[:, indices_f475x[3]]
        gamma2_chain_f475x = samples_mcmc[:, indices_f475x[4]]

        e_chain_f475x = np.sqrt(e1_chain_f475x**2 + e2_chain_f475x**2)
        q_mass_chain_f475x = (1 - e_chain_f475x) / (1 + e_chain_f475x)

        gamma_chain_f475x = np.sqrt(gamma1_chain_f475x**2 + gamma2_chain_f475x**2)
        phi_ext_chain_f475x = 0.5 * np.arctan2(e2_chain_f475x, e1_chain_f475x) * 180 / np.pi

        theta_E_median_f475x, theta_E_lower_f475x, theta_E_upper_f475x = get_median_and_uncertainties(theta_E_chain_f475x)
        q_mass_median_f475x, q_mass_lower_f475x, q_mass_upper_f475x = get_median_and_uncertainties(q_mass_chain_f475x)
        gamma_median_f475x, gamma_lower_f475x, gamma_upper_f475x = get_median_and_uncertainties(gamma_chain_f475x)
        phi_ext_median_f475x, phi_ext_lower_f475x, phi_ext_upper_f475x = get_median_and_uncertainties(phi_ext_chain_f475x)

        theta_E_f475x = f"${theta_E_median_f475x:.2f}^{{+{theta_E_upper_f475x:.2f}}}_{{-{theta_E_lower_f475x:.2f}}}$"
        q_mass_f475x = f"${q_mass_median_f475x:.2f}^{{+{q_mass_upper_f475x:.2f}}}_{{-{q_mass_lower_f475x:.2f}}}$"
        gamma_f475x = f"${gamma_median_f475x:.2f}^{{+{gamma_upper_f475x:.2f}}}_{{-{gamma_lower_f475x:.2f}}}$"
        phi_ext_f475x = f"${phi_ext_median_f475x:.1f}^{{+{phi_ext_upper_f475x:.1f}}}_{{-{phi_ext_lower_f475x:.1f}}}$"

    print(f"{name} & "
          f"{theta_E_f814w} & "
          f"{q_mass_f814w} & "
          f"{phi_ext_f814w} & "
          f"{gamma_f814w} & "
          f"{theta_E_f475x} & "
          f"{q_mass_f475x} & "
          f"{phi_ext_f475x} & "
          f"{gamma_f475x} \\\\")

print("\\hline")
print("\\end{tabular*}")
print("\\end{table*}")

\begin{table*}[htb]
\centering
\caption{Median lens mass parameters computed from the best-fit model. The associated uncertainties are statistical and were calculated using the 84th and 16th percentiles.}
\label{tab:lens_mass_params}
\begin{tabular*}{\textwidth}{@{\extracolsep{\fill}}lcccc|cccc}
\hline
\multicolumn{5}{c|}{{F814W}} & \multicolumn{4}{c}{{F475X}} \\
\hline
Lens System & $\theta_E$ [arcsec] & $q_{\text{mass}}$ & $\phi_{\text{mass}}$ [deg] & $\gamma$ & $\theta_E$ & $q_{\text{mass}}$ & $\phi_{\text{mass}}$ [deg] & $\gamma$ \\
\hline
J0407-5006 & $0.82^{+0.01}_{-0.01}$ & $0.80^{+0.07}_{-0.06}$ & $-68.1^{+15.3}_{-11.2}$ & $0.08^{+0.02}_{-0.03}$ & $1.00^{+0.01}_{-0.01}$ & $0.58^{+0.01}_{-0.01}$ & $32.4^{+1.4}_{-1.9}$ & $0.20^{+0.01}_{-0.01}$ \\
J0602-4335 & $1.00^{+0.01}_{-0.01}$ & $0.92^{+0.03}_{-0.05}$ & $-54.6^{+110.0}_{-19.3}$ & $0.11^{+0.02}_{-0.02}$ & $0.89^{+0.01}_{-0.01}$ & $0.58^{+0.03}_{-0.04}$ & $-80.7^{+3.2}_{-3.4}$ & $0.17^{+0.02}_{-0.01}$ \\
J0806+2006 & $0.75^{+0

In [2]:
import h5py
import numpy as np

# define filters and system names
names = ['J0407-5006', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
filter = 'F814W'

def mag_sigma(flux, sigma_flux):
    sigma_flux = np.asarray(sigma_flux)
    sigma_upper = sigma_flux[0]
    sigma_lower = sigma_flux[1]
    mag_err_upper = (2.5 / np.log(10)) * (sigma_upper / flux)
    mag_err_lower = (2.5 / np.log(10)) * (sigma_lower / flux)
    return mag_err_upper, mag_err_lower

print('FB / FA:')

# Table content
for name in names:
    filename = f"../../cutout_data/{name}/{filter}/{name}_{filter}_final.hdf5"
    with h5py.File(filename, "r") as f:
        kwargs = {key: f[key][()] for key in f}

    # Lens
    lens_flux = kwargs['lens_flux']
    lens_flux_sigma = kwargs['lens_flux_sigma']
    lens_mag = kwargs['lens_mag']
    lens_mag_sigmas = mag_sigma(lens_flux, lens_flux_sigma)

    # Image A
    im1_flux = kwargs['image_fluxes'][0]
    im1_flux_sigma = kwargs['image_fluxes_sigma'][0]
    im1_mag = kwargs['image_mags'][0]
    im1_mag_sigmas = mag_sigma(im1_flux, im1_flux_sigma)

    # Image B
    im2_flux = kwargs['image_fluxes'][1]
    im2_flux_sigma = kwargs['image_fluxes_sigma'][1]
    im2_mag = kwargs['image_mags'][1]
    im2_mag_sigmas = mag_sigma(im2_flux, im2_flux_sigma)

    print(f'{name}: {im2_flux / im1_flux:.3f}')

FB / FA:
J0407-5006: 3.567
J0806+2006: 0.470
J1442+4055: 0.466
J1515+1511: 0.690
J1620+1203: 3.975
J2325-5229: 0.689


J0806+2006: 0.520
J1442+4055: 0.422
J1620+1203: 3.841
J2325-5229: 0.560

In [1]:
import h5py
import numpy as np

# define filters and system names
names = ['J0806+2006', 'J1442+4055', 'J1620+1203', 'J2325-5229']
filter = 'F475X'

def mag_sigma(flux, sigma_flux):
    sigma_flux = np.asarray(sigma_flux)
    sigma_upper = sigma_flux[0]
    sigma_lower = sigma_flux[1]
    mag_err_upper = (2.5 / np.log(10)) * (sigma_upper / flux)
    mag_err_lower = (2.5 / np.log(10)) * (sigma_lower / flux)
    return mag_err_upper, mag_err_lower

print('FB / FA:')

# Table content
for name in names:
    filename = f"../../cutout_data/{name}/{filter}/{name}_{filter}_final.hdf5"
    with h5py.File(filename, "r") as f:
        kwargs = {key: f[key][()] for key in f}

    # Lens
    lens_flux = kwargs['lens_flux']
    lens_flux_sigma = kwargs['lens_flux_sigma']
    lens_mag = kwargs['lens_mag']
    lens_mag_sigmas = mag_sigma(lens_flux, lens_flux_sigma)

    # Image A
    im1_flux = kwargs['image_fluxes'][0]
    im1_flux_sigma = kwargs['image_fluxes_sigma'][0]
    im1_mag = kwargs['image_mags'][0]
    im1_mag_sigmas = mag_sigma(im1_flux, im1_flux_sigma)

    # Image B
    im2_flux = kwargs['image_fluxes'][1]
    im2_flux_sigma = kwargs['image_fluxes_sigma'][1]
    im2_mag = kwargs['image_mags'][1]
    im2_mag_sigmas = mag_sigma(im2_flux, im2_flux_sigma)

    print(f'{name}: {im2_flux / im1_flux:.3f}')

FB / FA:
J0806+2006: 0.520
J1442+4055: 0.423
J1620+1203: 3.856
J2325-5229: 0.560


# Fermat Potential and Time Delays

In [3]:
import h5py
import numpy as np

names_f814w = ['J0407-5006', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']

z_l = [0.55, 0.573, 0.284, 0.742, 0.398, 0.4]
z_s = [1.515, 1.54, 2.593, 2.049, 1.158, 2.739]

# LaTeX table header for F814W only
print("\\begin{table}[htb]")
print("\\centering")
print("\\caption{blah blah}")
print("\\label{tab:cosmo_results}")
print("\\begin{tabular}{lcccc}")
print("\\hline")
print("Lens System & $z_l$ & $z_s$ & $\\Delta \\Phi_{AB}$ [arcsec$\\^2$] & $\\Delta t_{AB}$ [days] \\\\")
print("\\hline")

def get_median_and_uncertainties(samples):
    median = np.percentile(samples, 50)
    lower = median - np.percentile(samples, 16)
    upper = np.percentile(samples, 84) - median
    return median, lower, upper

for i, name in enumerate(names_f814w):
    filename_f814w = f"../../cutout_data/{name}/F814W/{name}_F814W_cosmo.hdf5"

    with h5py.File(filename_f814w, "r") as f:
            kwargs = {}
            for key in f:
                kwargs[key] = f[key][()]

    dphi_chain = kwargs['dphi_list']
    dt_chain = kwargs['dt_list']

    dphi_median, dphi_lower, dphi_upper = get_median_and_uncertainties(dphi_chain)
    dt_median, dt_lower, dt_upper = get_median_and_uncertainties(dt_chain)

    z_l_fmt = f"${z_l[i]}$"
    z_s_fmt = f"${z_s[i]}$"
    dphi_fmt = f"${dphi_median:.3f}^{{+{dphi_upper:.3f}}}_{{-{dphi_lower:.3f}}}$"
    dt_fmt = f"${dt_median:.1f}^{{+{dt_upper:.1f}}}_{{-{dt_lower:.1f}}}$"

    print(f"{name} & {z_l_fmt} & {z_s_fmt} & {dphi_fmt} & {dt_fmt} \\\\")

print("\\hline")
print("\\end{tabular}")
print("\\end{table}")


\begin{table}[htb]
\centering
\caption{blah blah}
\label{tab:cosmo_results}
\begin{tabular}{lcccc}
\hline
Lens System & $z_l$ & $z_s$ & $\Delta \Phi_{AB}$ [arcsec$\^2$] & $\Delta t_{AB}$ [days] \\
\hline
J0407-5006 & $0.55$ & $1.515$ & $0.925^{+0.016}_{-0.028}$ & $99.5^{+1.7}_{-3.0}$ \\
J0806+2006 & $0.573$ & $1.54$ & $-0.468^{+0.023}_{-0.010}$ & $-53.3^{+2.6}_{-1.2}$ \\
J1442+4055 & $0.284$ & $2.593$ & $-0.655^{+0.007}_{-0.015}$ & $-25.8^{+0.3}_{-0.6}$ \\
J1515+1511 & $0.742$ & $2.049$ & $-1.188^{+0.020}_{-0.026}$ & $-174.9^{+3.0}_{-3.8}$ \\
J1620+1203 & $0.398$ & $1.158$ & $2.453^{+0.071}_{-0.080}$ & $183.2^{+5.3}_{-6.0}$ \\
J2325-5229 & $0.4$ & $2.739$ & $-0.712^{+0.031}_{-0.024}$ & $-41.5^{+1.8}_{-1.4}$ \\
\hline
\end{tabular}
\end{table}


In [4]:
import pickle
import numpy as np
import h5py

names_f814w = ['J0407-5006', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
names_f475x = ['J0806+2006', 'J1442+4055', 'J1620+1203', 'J2325-5229']

# LaTeX header
print("\\begin{table*}[htb]")
print("\\centering")
print("\\caption{Median lens light parameters computed from the best-fit model. The associated uncertainties are statistical and were calculated using the 84th and 16th percentiles.}")
print("\\label{tab:lens_light_params}")
print("\\begin{tabular*}{\\textwidth}{@{\\extracolsep{\\fill}}lcccc|cccc}")
print("\\hline")
print("\\multicolumn{5}{c|}{F814W} & \\multicolumn{4}{c}{F475X} \\\\")
print("\\hline")
print("Lens System & $R_{\\text{S\\'ersic}}$ [arcsec] & $n_{\\text{S\\'ersic}}$ & $q_L$ & $\\phi_L$ [$^\\circ$] & "
      "$R_{\\text{S\\'ersic}}$ [arcsec] & $n_{\\text{S\\'ersic}}$ & $q$ & $\\phi_L$ [$^\\circ$] \\\\")
print("\\hline")

def get_median_and_uncertainties(samples):
    median = np.percentile(samples, 50)
    lower = median - np.percentile(samples, 16)
    upper = np.percentile(samples, 84) - median
    return median, lower, upper

for i, name in enumerate(names_f814w):
    # --- F814W Data ---
    filename_f814w = f"../../cutout_data/{name}/F814W/{name}_F814W.pkl"
    with open(filename_f814w, "rb") as f:
        loaded_data_f814w = pickle.load(f)

    chain_list_f814w = loaded_data_f814w.get('chain_list')
    sampler_type, samples_mcmc_f814w, param_mcmc_f814w, dist_mcmc_f814w = chain_list_f814w[1]

    param_names = ['R_sersic_lens_light0', 'n_sersic_lens_light0', 'e1_lens_light0', 'e2_lens_light0']
    indices_f814w = [param_mcmc_f814w.index(p) for p in param_names]

    r_sersic_chain_f814w = samples_mcmc_f814w[:, indices_f814w[0]]
    n_sersic_chain_f814w = samples_mcmc_f814w[:, indices_f814w[1]]
    e1_chain_f814w = samples_mcmc_f814w[:, indices_f814w[2]]
    e2_chain_f814w = samples_mcmc_f814w[:, indices_f814w[3]]

    e_chain_f814w = np.sqrt(e1_chain_f814w**2 + e2_chain_f814w**2)
    q_chain_f814w = (1 - e_chain_f814w) / (1 + e_chain_f814w)
    theta_chain_f814w = 0.5 * np.arctan2(e2_chain_f814w, e1_chain_f814w) * 180 / np.pi

    r_median_f814w, r_lower_f814w, r_upper_f814w = get_median_and_uncertainties(r_sersic_chain_f814w)
    n_median_f814w, n_lower_f814w, n_upper_f814w = get_median_and_uncertainties(n_sersic_chain_f814w)
    q_median_f814w, q_lower_f814w, q_upper_f814w = get_median_and_uncertainties(q_chain_f814w)
    theta_median_f814w, theta_lower_f814w, theta_upper_f814w = get_median_and_uncertainties(theta_chain_f814w)

    theta_str_f814w = f"${theta_median_f814w:.1f}^{{+{theta_upper_f814w:.1f}}}_{{-{theta_lower_f814w:.1f}}}^\\circ$"

    # --- F475X Data (only if available) ---
    if name in names_f475x:
        j = names_f475x.index(name)
        filename_f475x = f"../../cutout_data/{name}/F475X/{name}_F475X.pkl"
        with open(filename_f475x, "rb") as f:
            loaded_data_f475x = pickle.load(f)

        chain_list_f475x = loaded_data_f475x.get('chain_list')
        sampler_type, samples_mcmc_f475x, param_mcmc_f475x, dist_mcmc_f475x = chain_list_f475x[1]

        indices_f475x = [param_mcmc_f475x.index(p) for p in param_names]

        r_sersic_chain_f475x = samples_mcmc_f475x[:, indices_f475x[0]]
        n_sersic_chain_f475x = samples_mcmc_f475x[:, indices_f475x[1]]
        e1_chain_f475x = samples_mcmc_f475x[:, indices_f475x[2]]
        e2_chain_f475x = samples_mcmc_f475x[:, indices_f475x[3]]

        e_chain_f475x = np.sqrt(e1_chain_f475x**2 + e2_chain_f475x**2)
        q_chain_f475x = (1 - e_chain_f475x) / (1 + e_chain_f475x)
        theta_chain_f475x = 0.5 * np.arctan2(e2_chain_f475x, e1_chain_f475x) * 180 / np.pi

        r_median_f475x, r_lower_f475x, r_upper_f475x = get_median_and_uncertainties(r_sersic_chain_f475x)
        n_median_f475x, n_lower_f475x, n_upper_f475x = get_median_and_uncertainties(n_sersic_chain_f475x)
        q_median_f475x, q_lower_f475x, q_upper_f475x = get_median_and_uncertainties(q_chain_f475x)
        theta_median_f475x, theta_lower_f475x, theta_upper_f475x = get_median_and_uncertainties(theta_chain_f475x)

        r_str_f475x = f"${r_median_f475x:.2f}^{{+{r_upper_f475x:.2f}}}_{{-{r_lower_f475x:.2f}}}$"
        n_str_f475x = f"${n_median_f475x:.2f}^{{+{n_upper_f475x:.2f}}}_{{-{n_lower_f475x:.2f}}}$"
        q_str_f475x = f"${q_median_f475x:.3f}^{{+{q_upper_f475x:.3f}}}_{{-{q_lower_f475x:.3f}}}$"
        theta_str_f475x = f"${theta_median_f475x:.1f}^{{+{theta_upper_f475x:.1f}}}_{{-{theta_lower_f475x:.1f}}}^\\circ$"
    else:
        r_str_f475x = "$--$"
        n_str_f475x = "$--$"
        q_str_f475x = "$--$"
        theta_str_f475x = "$--$"

    # Print table row
    print(f"{name} & "
          f"${r_median_f814w:.2f}^{{+{r_upper_f814w:.2f}}}_{{-{r_lower_f814w:.2f}}}$ & "
          f"${n_median_f814w:.2f}^{{+{n_upper_f814w:.2f}}}_{{-{n_lower_f814w:.2f}}}$ & "
          f"${q_median_f814w:.3f}^{{+{q_upper_f814w:.3f}}}_{{-{q_lower_f814w:.3f}}}$ & "
          f"{theta_str_f814w} & "
          f"{r_str_f475x} & {n_str_f475x} & {q_str_f475x} & {theta_str_f475x} \\\\")

print("\\hline")
print("\\end{tabular*}")
print("\\end{table*}")


\begin{table*}[htb]
\centering
\caption{Median lens light parameters computed from the best-fit model. The associated uncertainties are statistical and were calculated using the 84th and 16th percentiles.}
\label{tab:lens_light_params}
\begin{tabular*}{\textwidth}{@{\extracolsep{\fill}}lcccc|cccc}
\hline
\multicolumn{5}{c|}{F814W} & \multicolumn{4}{c}{F475X} \\
\hline
Lens System & $R_{\text{S\'ersic}}$ [arcsec] & $n_{\text{S\'ersic}}$ & $q_L$ & $\phi_L$ [$^\circ$] & $R_{\text{S\'ersic}}$ [arcsec] & $n_{\text{S\'ersic}}$ & $q$ & $\phi_L$ [$^\circ$] \\
\hline
J0407-5006 & $0.40^{+0.01}_{-0.01}$ & $4.85^{+0.15}_{-0.13}$ & $0.664^{+0.009}_{-0.009}$ & $-8.3^{+0.9}_{-0.9}^\circ$ & $--$ & $--$ & $--$ & $--$ \\
J0806+2006 & $0.52^{+0.01}_{-0.01}$ & $3.87^{+0.10}_{-0.09}$ & $0.937^{+0.010}_{-0.010}$ & $73.6^{+4.2}_{-4.2}^\circ$ & $0.24^{+0.03}_{-0.06}$ & $1.45^{+0.32}_{-0.38}$ & $0.695^{+0.051}_{-0.046}$ & $18.7^{+5.5}_{-6.1}^\circ$ \\
J1442+4055 & $0.94^{+0.01}_{-0.01}$ & $5.97^{+0.02}_{-0.04